In [5]:
import torch
#from ImageDataLoader import DataLoader


SyntaxError: invalid syntax (<ipython-input-5-607be7319a13>, line 3)

In [6]:
train_data = DataLoader(trainpath, args.data+'/train.txt',
                            classes=args.classes)

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                            batch_size=args.batch,
                                            shuffle=True,
                                            num_workers=args.cores)

NameError: name 'DataLoader' is not defined

In [7]:
for i, (images, labels, original) in enumerate(train_loader):

SyntaxError: invalid syntax (<ipython-input-7-4d9ba2a11c87>, line 1)